In [1]:
import getpass
import os

if "AZURE_OPENAI_API_KEY" not in os.environ:
    os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass(
        "Enter your AzureOpenAI API key: "
    )
os.environ["AZURE_OPENAI_ENDPOINT"]="https://hassanllms.openai.azure.com/"

In [2]:
LANGCHAIN_TRACING_V2="true"
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_API_KEY="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"
LANGCHAIN_PROJECT="pr-aching-chess-83"


from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment="gpt-4o-mini",  # or your deployment
    api_version="2024-02-15-preview",  # or your api version
    temperature=0,
    # other params...
)

In [5]:
import pandas as pd

df = pd.read_csv("MaturityCheck.csv")
print(df.shape)
print(df.columns.tolist())

(46, 3)
['Department', 'Category', 'Question']


In [10]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///CompanyDB_test.db")

In [7]:
df.to_sql("CompanyDBTable1", engine, index=False)

46

In [11]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['CompanyDBTable1']


In [12]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002A5225C7AD0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002A5225C7AD0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002A5225C7AD0>),
 QuerySQLCheckerTool(description='Use this tool to 

In [14]:
from langchain_core.messages import SystemMessage

# SQL_PREFIX = """You are an agent designed to interact with a SQL database.
# Given an input , create a syntactically correct sqlite query to run, then look at the results of the query and return the answer.You are an intelligent interview bot conducting a maturity assessment. You have access to a SQLite database with a 'MaturityTable' table (columns: Department, Category, Question).


# IMPORTANT: You must ONLY ask questions that were retrieved from the database. Do not generate new questions.

# IMPORTANT INSTRUCTIONS:
# 1. When you first interact with a user:
#    - Extract their name, email, and department from their message
#    - Use this SQL query to get all questions for their department:
#      SELECT Category, Question FROM MaturityCheck WHERE Department = ? ORDER BY Category;
#    - Store these questions in your context for the conversation

# 2. After getting the questions:
#    - Ask questions one at a time
#    - Wait for the user's response before asking the next question
#    - Group questions by category and introduce each category
#    - Keep track of which questions you've asked
#    - Don't repeat questions that have been answered

# 3. For each response:
#    - Briefly acknowledge the answer
#    - Move to the next question
#    - When finishing a category, introduce the next one"""

SQL_PREFIX = """You are an intelligent assessment bot designed to conduct a maturity assessment for different departments. You'll guide users through a series of questions to evaluate their department's maturity level.

Initial Greeting:
When a user first connects, introduce yourself and explain:
1. The purpose of this assessment (evaluating department maturity)
2. What to expect (series of questions grouped by categories)
3. That their responses will be saved for analysis
4. Ask for their name, email, and department to begin

Database Instructions:
You have access to a SQLite database with a 'MaturityDBTable1' table containing department-specific questions. Only ask questions from this database.

IMPORTANT WORKFLOW:
1. First Interaction:
   - Warmly greet the user and explain the process
   - Extract their name, email, and department
   - Use this SQL query to get their department's questions:
     SELECT Category, Question FROM MaturityDBTable1 WHERE Department = ? ORDER BY Category;
   - Store these questions for the conversation

2. Question Flow:
   - Before starting questions, explain which categories will be covered
   - Group questions by category
   - Introduce each new category before its questions
   - Ask one question at a time
   - Wait for user response before proceeding
   - Keep track of asked questions to avoid repetition

3. Response Handling:
   - Acknowledge each answer briefly
   - Transition smoothly to the next question
   - When switching categories, announce the new category
   - At the end, thank the user and explain that their responses have been recorded

Remember to maintain a professional yet friendly tone throughout the assessment."""


system_message = SystemMessage(content=SQL_PREFIX)

In [9]:
%%capture --no-stderr
%pip install --upgrade --quiet langgraph

In [15]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

agent_executor = create_react_agent(llm, tools, messages_modifier=system_message,
                                    checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}


C:\Users\Hamza\AppData\Local\Temp\ipykernel_3332\3246979625.py:8: LangGraphDeprecationWarning: Parameter 'messages_modifier' in function 'create_react_agent' is deprecated as of version 0.1.9 and will be removed in version 0.3.0. Use 'state_modifier' parameter instead.
  agent_executor = create_react_agent(llm, tools, messages_modifier=system_message,


In [20]:
config = {"configurable": {"thread_id": "abc123"}}


query = "my name is Bhatti, and I am from the technical architecture department, my email is batt@gmail.com"
print(agent_executor.checkpointer.__dict__)
for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

# result = agent_executor.stream(
#     {"messages": [HumanMessage(content="Hi, my name is Abu bakar having email address abu@gmail.com, and I belong to Technical Architecture department")]}
# )

{'serde': <langgraph.checkpoint.serde.jsonplus.JsonPlusSerializer object at 0x000002A52268F770>, 'storage': defaultdict(<function MemorySaver.__init__.<locals>.<lambda> at 0x000002A51F16BD80>, {'abc123': defaultdict(<class 'dict'>, {'': {'1ef94e5c-f2e0-6482-bfff-222f85b08f8e': (('msgpack', b'\x86\xa1v\x01\xa2ts\xd9 2024-10-28T04:33:41.609587+00:00\xa2id\xd9$1ef94e5c-f2e0-6482-bfff-222f85b08f8e\xaechannel_values\x81\xa9__start__\x81\xa8messages\x91\xc7\xf1\x05\x94\xbdlangchain_core.messages.human\xacHumanMessage\x87\xa7content\xd9bmy name is Bhatti, and I am from the technical architecture department, my email is batt@gmail.com\xb1additional_kwargs\x80\xb1response_metadata\x80\xa4type\xa5human\xa4name\xc0\xa2id\xc0\xa7example\xc2\xb3model_validate_json\xb0channel_versions\x81\xa9__start__\xd9300000000000000000000000000000001.0.6022676831889675\xadversions_seen\x81\xa9__input__\x80'), ('msgpack', b'\x84\xa6source\xa5input\xa6writes\x81\xa9__start__\x81\xa8messages\x91\xc7\xf1\x05\x94\xbd

In [21]:
print(s)

{'agent': {'messages': [AIMessage(content="I've successfully retrieved the questions for the technical architecture department. \n\n### Categories Covered:\n1. **Data**\n2. **Application**\n\nLet's start with the first category: **Data**.\n\n**Question 1:**  \nDo you have any data Architecture Repository? Are data flows through the network known and documented?\n\nPlease provide your response!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 1456, 'total_tokens': 1523, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_d54531d9eb', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': F

In [18]:
question_content = s['agent']['messages'][0].content

question_content

"Hello Bhatti! Thank you for connecting. I'm here to help you evaluate the maturity of your department through a series of questions.\n\n### Purpose of the Assessment:\n1. **Evaluate Department Maturity**: We'll assess various aspects of your department's processes and practices.\n2. **What to Expect**: You'll answer a series of questions grouped by categories.\n3. **Data Privacy**: Your responses will be saved for analysis, and your information will be kept confidential.\n\n### Categories Covered:\nI'll be asking questions from different categories related to the technical architecture department.\n\nLet's get started! \n\n**Category 1: [Insert Category Name Here]**  \nHere's your first question:  \n[Insert Question Here]  \n\nPlease provide your response!"

In [19]:
s['agent']['messages'][0]['content']

TypeError: 'AIMessage' object is not subscriptable

In [12]:
config = {"configurable": {"thread_id": "abc123"}}


query = "Yes, we have applicaiton repository, and all applications implemented are known."

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")


{'agent': {'messages': [AIMessage(content="Thank you for your response! \n\nHere's the next question in the **Application** category:\n\n**2. Are specialized applications experts assigned accordingly? Is there an implementation and life cycle process for new applications, and 2nd and 3rd level support?** \n\nWhat are your thoughts on this?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 1174, 'total_tokens': 1235, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_878413d04d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-d1173018-3807-

In [13]:
pip freeze

aiohappyeyeballs==2.4.3
aiohttp==3.10.10
aiosignal==1.3.1
altair==5.4.1
annotated-types==0.7.0
anyio==4.6.2.post1
asttokens==2.4.1
attrs==24.2.0
blinker==1.8.2
cachetools==5.5.0
certifi==2024.8.30
charset-normalizer==3.4.0
click==8.1.7
colorama==0.4.6
comm==0.2.2
dataclasses-json==0.6.7
debugpy==1.8.7
decorator==5.1.1
distro==1.9.0
executing==2.1.0
fastapi==0.115.3
frozenlist==1.4.1
gitdb==4.0.11
GitPython==3.1.43
greenlet==3.1.1
h11==0.14.0
httpcore==1.0.6
httpx==0.27.2
httpx-sse==0.4.0
idna==3.10
ipykernel==6.29.5
ipython==8.28.0
jedi==0.19.1
Jinja2==3.1.4
jiter==0.6.1
jsonpatch==1.33
jsonpointer==3.0.0
jsonschema==4.23.0
jsonschema-specifications==2024.10.1
jupyter_client==8.6.3
jupyter_core==5.7.2
langchain==0.3.4
langchain-community==0.3.3
langchain-core==0.3.12
langchain-openai==0.2.3
langchain-text-splitters==0.3.0
langgraph==0.2.39
langgraph-checkpoint==2.0.1
langgraph-sdk==0.1.33
langsmith==0.1.136
markdown-it-py==3.0.0
MarkupSafe==3.0.2
marshmallow==3.23.0
matplotlib-inline==

In [47]:
query = "Yes, spedialized applicaitons expterts are assinged accordinlgly.Life cycle is 3 months, we do have even 4th level support."

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content="Thank you for your response! It's good to know that specialized application experts are assigned and that you have a structured life cycle and support levels in place.\n\nNow, let's transition to the **Data** category. Here’s the first question:\n\n**Data:**  \nDo you have any data Architecture Repository? Are data flows through the network known and documented?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 1007, 'total_tokens': 1077, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_878413d04d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'd

In [48]:
query = "Yes. we do have architecture repository, and data flows through the network are known and are also documentned"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content="That's excellent! Having a data architecture repository and documented data flows is crucial for effective data management.\n\nLet's move on to the next question in the **Data** category:\n\n**Data:**  \nAre all IT employees aware of data sensitivity? How is Data Management & Data Governance conducted?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 1105, 'total_tokens': 1161, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_878413d04d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-980fdf0f-

In [49]:
query = "Yes, all employees are aware of the data sensitiviy and it is managed by a third party secure partner."

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content="Thank you for your response! It's reassuring to know that all IT employees are aware of data sensitivity and that data management is handled by a secure third-party partner.\n\nNow, let's move on to the **Infrastructure** category. Here’s the first question:\n\n**Infrastructure:**  \nDoes Infrastructure Architecture follow pre-defined blueprints and is it documented?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 1190, 'total_tokens': 1258, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_878413d04d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': Fals

In [50]:
query = "Yes, Infrastructure Architecture follow pre-defined blueprints and is documented?"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='Great to hear that your Infrastructure Architecture follows pre-defined blueprints and is well documented!\n\nNow, here’s the next question in the **Infrastructure** category:\n\n**Infrastructure:**  \nHow is Infrastructure Management conducted? Does it follow international standards? Are there any standardization measures?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 1278, 'total_tokens': 1333, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_878413d04d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'

In [51]:
query = "Yes, it follows international standards by applying all the standard rules. and there are many measures to track the overall performance of the architecture."

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content="Thank you for your response! It's impressive that your Infrastructure Management adheres to international standards and has measures in place to track performance.\n\nThat concludes our maturity assessment for the Technical Architecture department. If you have any further questions or need assistance, feel free to ask!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 1367, 'total_tokens': 1420, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_878413d04d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, 

In [2]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Hi, my name is Abu bakar having email address abu@gmail.com, and I belong to Technical Architecture department")]}
):
    print(s)
    print("----")

NameError: name 'agent_executor' is not defined

In [15]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Yes, we have application architecture repo, and all applications are implemented are known.")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content="It seems like you're discussing your application architecture. To proceed with the maturity assessment, could you please provide me with your name, email, and department?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 593, 'total_tokens': 624, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_878413d04d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-5c322427-f251-4f03-8591-f49ab9aa380c-0', usage_metadata={'input_tokens': 593, 'output_tokens': 31, 'total_tokens': 624, 'input_token_details': {

In [ ]:
agent.stream(
    {"messages": [HumanMessage(content="How many albums does alis in chain have?")]}
):

In [47]:
# Your existing imports
import requests
from langchain_openai import AzureChatOpenAI
# ... (rest of your imports)

# Your existing code until the agent executor setup
# ...

# Modified agent execution code
def process_chat(query: str):
    # Initial message from user
    query = query
    
    # First, create user in database
    user_response = requests.post(
        "http://localhost:8000/users/",
        json={"message": query}
    ).json()
    user_id = user_response["user_id"]
    
    # Now start the conversation
    for s in agent_executor.stream(
        {"messages": [HumanMessage(content=query)]}, 
        config=config
    ):
        if 'agent' in s and 'messages' in s['agent']:
            message = s['agent']['messages'][0]
            
            # If this is a question from the bot (not intermediate processing)
            if message.content and not message.additional_kwargs.get('tool_calls'):
                print("Bot:", message.content)
                
                # Get user's answer (in real scenario, this would come from user input)
                user_answer = input("Your answer: ")
                
                # Save the response
                response = requests.post(
                    "http://localhost:8000/responses/",
                    json={
                        "user_id": user_id,
                        "bot_message": message.content,
                        "user_answer": user_answer
                    }
                )



In [ ]:
# Your existing imports
import requests
from langchain_openai import AzureChatOpenAI
# ... (rest of your imports)

# Your existing code until the agent executor setup
# ...

# Modified agent execution code
def process_chat(query: str):
    # Initial message from user
    query = query
    
    # First, create user in database
    user_response = requests.post(
        "http://localhost:8000/users/",
        json={"message": query}
    ).json()
    user_id = user_response["user_id"]
    
    # Now start the conversation
    for s in agent_executor.stream(
        {"messages": [HumanMessage(content=query)]}, 
        config=config
    ):
        if 'agent' in s and 'messages' in s['agent']:
            message = s['agent']['messages'][0]
            
            # If this is a question from the bot (not intermediate processing)
            if message.content and not message.additional_kwargs.get('tool_calls'):
                print("Bot:", message.content)
                
                # Get user's answer (in real scenario, this would come from user input)
                user_answer = input("Your answer: ")
                
                # Save the response
                response = requests.post(
                    "http://localhost:8000/responses/",
                    json={
                        "user_id": user_id,
                        "bot_message": message.content,
                        "user_answer": user_answer
                    }
                )




In [48]:
# Run the chat process
process_chat("Hi, my name is Hamza having email address hamza@gmail.com, and I belong to Technical Architecture department")

Bot: Thank you, Hamza! Let's begin the maturity assessment for the Technical Architecture department. 

We'll start with the **Application** category. Here’s the first question:

**Application:**  
Do you have any application Architecture Repository? Are all applications implemented known and documented? Are self-developed applications in place? If yes, how are these managed?
